# Analyse 1 : utilisation de regexp et de patterns spacy

on essaie d'extraire des termes connus, et des locutions typiques, pour détecter les références aux personnages et aux lieux.



## Trouver et lister les fichiers

In [7]:
from glob import glob
import os.path

In [8]:
PATH_TEXT_FILES_DIR = "/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts"

In [10]:
files = sorted(glob(os.path.join(PATH_TEXT_FILES_DIR, "*.txt")))
print("Found", len(files), "files.")
files[:10]

Found 100 files.


['/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00101_Adam.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00102_Adam.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00201_Audoux.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00301_Aimard.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00302_Aimard.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00401_Allais.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00501_Balzac.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00502_Balzac.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00503_Balzac.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00601_Boisgobey.txt']

In [11]:
sample = files[50]
sample

'/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA03001_Ohnet.txt'

## Import spacy and start processing

In [ ]:
import spacy


## L'objet `nlp`
On construit une nouvelle chaîne de traitements de plusieurs façon. La manière la plus simple est de construire une chaîne de traitement vide (ou presque) pour le français à l'aide de la "fabrique" à chaînes de traitement `spacy.blank(LANGAGE)`.

In [ ]:
nlp = spacy.blank("fr")


La chaîne de traitements contient différents traitements appliqués les uns après les autres.
On peut afficher cette liste de traitements à l'aide de l'attribut `pipe_names` de l'objet `nlp`.

In [13]:
nlp.pipe_names

[]

Par défaut, une chaîne de traitement ne contient rien… Sauf un *tokenizer*, d'où l'importance de préciser la langue !

## L'objet `doc`
On obtient un objet `doc` en appliquant la chaîne de traitement `nlp` à une chaîne de texte.


In [16]:
# Créé en traitant une chaine de caractères avec l'objet nlp
doc = nlp("Bonjour tout le monde !")

Cet objet `doc` est central pour Spacy : il contient toute les informations produite par la chaîne de traitement à propos de notre texte.

On peut parcourir les *tokens* extraits à l'aide d'une boucle classique en Python.

In [17]:
# Itère sur les tokens dans un Doc
for token in doc:
    print(token.text)

Bonjour
tout
le
monde
!


Lorsqu'on souhaite traiter plusieurs documents, on peut utiliser `nlp.pipe(LISTE_DE_TEXTES)`.
Dans ce cas, on obtient une liste de documents en sortie, qu'il est possible d'inspecter avec une seconde boucle !

In [18]:
TEXTES = [
    "Bonjour tout le monde !", 
    "Comment allez-vous ?",
    "Savez-vous qu'une chaîne de caractères peut contenir des retours à la ligne\ncomme celui-ci ?"
    ]
documents = nlp.pipe(TEXTES)
for doc_id, doc in enumerate(documents):
    for token_id, token in enumerate(doc):
        print(f"doc#{doc_id}, tok#{token_id}: {token.text}")

doc#0, tok#0: Bonjour
doc#0, tok#1: tout
doc#0, tok#2: le
doc#0, tok#3: monde
doc#0, tok#4: !
doc#1, tok#0: Comment
doc#1, tok#1: allez
doc#1, tok#2: -vous
doc#1, tok#3: ?
doc#2, tok#0: Savez
doc#2, tok#1: -vous
doc#2, tok#2: qu'
doc#2, tok#3: une
doc#2, tok#4: chaîne
doc#2, tok#5: de
doc#2, tok#6: caractères
doc#2, tok#7: peut
doc#2, tok#8: contenir
doc#2, tok#9: des
doc#2, tok#10: retours
doc#2, tok#11: à
doc#2, tok#12: la
doc#2, tok#13: ligne
doc#2, tok#14: 

doc#2, tok#15: comme
doc#2, tok#16: celui-ci
doc#2, tok#17: ?
